In [17]:
import torch
import triton
import triton.backends
import triton.language as tl
import math
import os
from flash_mla import get_mla_metadata, flash_mla_with_kvcache
from mla import triton_mqa
from flash_attn_interface import flash_attn_with_kvcache
os.environ['TRITON_PRINT_AUTOTUNING'] = '1'
triton.__version__

'3.1.0'

In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda')
dtype = torch.bfloat16
bs, num_head, q_len, kv_len, rope_head_dim, nope_head_dim=32, 128, 1, 8192, 64, 128
kv_lora_rank = 512
scale = (rope_head_dim + nope_head_dim) ** (-0.5)
q = torch.randn(bs, num_head, q_len, kv_lora_rank+rope_head_dim, device=device, dtype=dtype)
k = torch.randn(bs, 1, kv_len, kv_lora_rank+rope_head_dim, device=device, dtype=dtype)
v = torch.randn(bs, 1, kv_len, kv_lora_rank, device=device, dtype=dtype)
k[..., :kv_lora_rank] = v
# attention_mask = torch.ones(bs, kv_len, device=device, dtype=torch.int32)
# # attention_mask[:, :10] = 0
# attention_mask[:, :100] = 0

cache_seqlens = torch.full((bs,), kv_len, dtype=torch.int32, device=device)
total_seqlens = cache_seqlens.sum().item()
mean_seqlens = cache_seqlens.float().mean().int().item()
max_seqlen = cache_seqlens.max().item()
max_seqlen_pad = triton.cdiv(max_seqlen, 256) * 256
block_size = 64
block_table = torch.arange(bs * max_seqlen_pad // block_size, dtype=torch.int32, device=device).view(bs, max_seqlen_pad // block_size)
blocked_k = k.view(block_table.numel(), block_size, 1, kv_lora_rank+rope_head_dim)
# for i in range(bs):
#     blocked_k.view(bs, max_seqlen_pad, 1, kv_lora_rank+rope_head_dim)[i, cache_seqlens[i].item():] = float("nan")
# blocked_v = blocked_k[..., :kv_lora_rank]
tile_scheduler_metadata, num_splits = get_mla_metadata(cache_seqlens, q_len * num_head // 1, 1)



In [13]:
out1 = triton_mqa(q, k, v, scale, attention_mask=None)

In [14]:
out2, lse = flash_mla_with_kvcache(
            q, blocked_k, block_table, cache_seqlens, kv_lora_rank,
            tile_scheduler_metadata, num_splits, softmax_scale=scale, causal=True)

In [19]:
triton.testing.do_bench(lambda: triton_mqa(q, k, v, scale, attention_mask=None))

0.5333253145217896

In [20]:
triton.testing.do_bench(lambda: flash_mla_with_kvcache(
            q, blocked_k, block_table, cache_seqlens, kv_lora_rank,
            tile_scheduler_metadata, num_splits, softmax_scale=scale, causal=True))

0.15365096926689148

In [ ]:
@torch.inference_mode()
def test_flash_mla(b, s_q, mean_sk, h_q, h_kv, d, dv, causal, varlen):
    print(f"{b=}, {s_q=}, {mean_sk=}, {h_q=}, {h_kv=}, {d=}, {dv=}, {causal=}, {varlen=}")

    cache_seqlens = torch.full((b,), mean_sk, dtype=torch.int32)
    if varlen:
        for i in range(b):
            cache_seqlens[i] = max(random.normalvariate(mean_sk, mean_sk / 2), s_q)
    total_seqlens = cache_seqlens.sum().item()
    mean_seqlens = cache_seqlens.float().mean().int().item()
    max_seqlen = cache_seqlens.max().item()
    max_seqlen_pad = triton.cdiv(max_seqlen, 256) * 256
    # print(f"{total_seqlens=}, {mean_seqlens=}, {max_seqlen=}")

    q = torch.randn(b, s_q, h_q, d)
    block_size = 64
    block_table = torch.arange(b * max_seqlen_pad // block_size, dtype=torch.int32).view(b, max_seqlen_pad // block_size)
    blocked_k = torch.randn(block_table.numel(), block_size, h_kv, d)
    for i in range(b):
        blocked_k.view(b, max_seqlen_pad, h_kv, d)[i, cache_seqlens[i].item():] = float("nan")
    blocked_v = blocked_k[..., :dv]

    tile_scheduler_metadata, num_splits = get_mla_metadata(cache_seqlens, s_q * h_q // h_kv, h_kv)

    def flash_mla():
        return flash_mla_with_kvcache(
            q, blocked_k, block_table, cache_seqlens, dv,
            tile_scheduler_metadata, num_splits, causal=causal,
        )

    def ref_mla():
        out = torch.empty(b, s_q, h_q, dv, dtype=torch.float32)
        lse = torch.empty(b, h_q, s_q, dtype=torch.float32)
        for i in range(b):
            begin = i * max_seqlen_pad
            end = begin + cache_seqlens[i]
            O, LSE = scaled_dot_product_attention(
                q[i].transpose(0, 1),
                blocked_k.view(-1, h_kv, d)[begin:end].transpose(0, 1),
                blocked_v.view(-1, h_kv, dv)[begin:end].transpose(0, 1),
                h_q=h_q,
                h_kv=h_kv,
                is_causal=causal,
            )
            out[i] = O.transpose(0, 1)
            lse[i] = LSE
        return out, lse

    out_flash, lse_flash = flash_mla()
    out_torch, lse_torch = ref_mla()
    cal_diff(out_flash, out_torch, "out")
    cal_diff(lse_flash, lse_torch, "lse")

    t = triton.testing.do_bench(flash_mla)
    FLOPS = s_q * total_seqlens * h_q * (d + dv) * 2
    bytes = (total_seqlens * h_kv * d + b * s_q * h_q * d + b * s_q * h_q * dv) * (torch.finfo(dtype).bits // 8)
    print(f"{t:.3f} ms, {FLOPS / 10 ** 9 / t:.0f} TFLOPS, {bytes / 10 ** 6 / t:.0f} GB/s")